In [6]:
import os
import pandas as pd

# Define the path to the root folder containing the result files
root_folder_path = '/gpfs/commons/groups/gursoy_lab/shong/Github/composable-fe-rs/out060824/i_i_1'

# Initialize lists to store the extracted data
dim0_list = []
dim1_list = []
dim2_list = []
setup_list = []
keygen_list = []
enc_list = []
dec_list = []

# Function to extract the necessary information from a single file
def extract_data_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # Extract parameters at the start of the file
        dim0 = dim1 = dim2 = None
        for line in lines:
            if line.startswith('dim0'):
                dim0 = int(line.split(':')[1].strip())
            elif line.startswith('dim1'):
                dim1 = int(line.split(':')[1].strip())
            elif line.startswith('dim2'):
                dim2 = int(line.split(':')[1].strip())
        
        # Extract time summaries at the end of the file
        setup_time = keygen_time = enc_time = dec_time = None
        for line in lines:
            if line.startswith('SETUP:'):
                setup_time = parse_time(line.split(':')[1].strip())
            elif line.startswith('KEYGEN:'):
                keygen_time = parse_time(line.split(':')[1].strip())
            elif line.startswith('ENC:'):
                enc_time = parse_time(line.split(':')[1].strip())
            elif line.startswith('DEC:'):
                dec_time = parse_time(line.split(':')[1].strip())
        
        return dim0, dim1, dim2, setup_time, keygen_time, enc_time, dec_time

# Function to parse and convert time strings to seconds
def parse_time(time_str):
    if time_str.endswith('ms'):
        return float(time_str.replace('ms', '')) / 1000.0
    elif time_str.endswith('s'):
        return float(time_str.replace('s', ''))
    else:
        raise ValueError(f"Unknown time format: {time_str}")

# Function to recursively search through all subdirectories and process the files
def process_all_files(root_folder):
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.txt'):
                file_path = os.path.join(dirpath, filename)
                dim0, dim1, dim2, setup, keygen, enc, dec = extract_data_from_file(file_path)
                
                # Append the extracted data to the lists
                dim0_list.append(dim0)
                dim1_list.append(dim1)
                dim2_list.append(dim2)
                setup_list.append(setup)
                keygen_list.append(keygen)
                enc_list.append(enc)
                dec_list.append(dec)

# Process all files in the root folder and its subdirectories
process_all_files(root_folder_path)

# Create a DataFrame from the extracted data
data = {
    'dim0': dim0_list,
    'dim1': dim1_list,
    'dim2': dim2_list,
    'SETUP': setup_list,
    'KEYGEN': keygen_list,
    'ENC': enc_list,
    'DEC': dec_list
}
df = pd.DataFrame(data)

# sort by dim0
df = df.sort_values(by=['dim0', 'dim1', 'dim2'])
# chagne KEYGEN and DEC to hour
df['KEYGEN'] = df['KEYGEN'] / 3600
df['DEC'] = df['DEC'] / 3600
df.head()

,dim0,dim1,dim2,SETUP,KEYGEN,ENC,DEC
5,1,1,1,1.445318,0.648150,0.937909,0.160036
2,2,2,1,2.235875,1.408728,0.943960,0.391659
1,3,3,1,3.280576,2.518764,0.919395,1.106367
0,4,4,1,4.744633,3.987845,0.939768,1.235601
4,5,5,1,6.639810,5.914387,1.137036,3.181922
